In [2]:
from google.cloud.bigquery.client import Client
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
proyect_id = "peya-data-analyt-factory-stg"
client = Client(project=proyect_id)

/home/creyesp/Projects/ml-workshop-campus-party/venv/lib/python3.8/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
query_path = "../data/queries/floot_it_dataset.sql"
with open(query_path, "r") as file:
    query = "".join(file.readlines())

full_dataset = client.query(query).to_dataframe(create_bqstorage_client=False)

In [5]:
output_path = "../data/users_raw.csv"
full_dataset.to_csv(output_path, index=False)
full_dataset.drop(columns=['split_group'], inplace=True)

In [6]:
full_dataset['user_first_engagement'].dt.day

0         4
1         7
2         5
3        25
4        21
         ..
13583    14
13584    12
13585     2
13586    19
13587    14
Name: user_first_engagement, Length: 13588, dtype: int64

In [10]:
selector = (full_dataset["is_enable"] == 1) & (full_dataset["bounced"] == 0)

train, test = train_test_split(
    full_dataset.loc[selector],
    test_size=0.1,
    random_state=42,
    shuffle=True,
    stratify=full_dataset.loc[selector, "churned"],
    # stratify=full_dataset.loc[selector, 'user_first_engagement'].dt.dayofyear,
)

train_path = "../data/users_train.csv"
test_path = "../data/users_test.csv"
train.drop(columns=['is_enable', 'bounced']).to_csv(train_path, index=False)
test.drop(columns=['is_enable', 'bounced']).to_csv(test_path, index=False)

In [11]:
train['churned'].mean(), test['churned'].mean()

(0.2310152990264256, 0.23153942428035043)